## 01 LCEL(Langchain Expression Language): Basic Example


In [2]:

from langchain.chat_models import init_chat_model
import os
from dotenv import load_dotenv
load_dotenv()
llm = init_chat_model(
    model="deepseek-chat",
    model_provider="openai",
    api_key=os.environ.get('DS_API_KEY'),
    base_url="https://api.deepseek.com",
    temperature = 0.2,
)

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}."),
        ("human", "Tell me {joke_count} jokes in {language}."),
    ]
)
chain = prompt_template | llm | StrOutputParser()
result = chain.invoke({"topic": "dogs", "joke_count": 3, "language": "English"})


In [3]:
from IPython.display import Markdown
Markdown(result)

Sure, here are three dog-themed jokes for you:

1. **Why did the dog sit in the shade?**  
   Because he didn't want to be a hot dog!

2. **What do you call a dog magician?**  
   A labracadabrador!

3. **Why don’t dogs make good secret keepers?**  
   Because they wag their tails!

## 02 LCEL: Under the Hood

In [4]:
from langchain.schema.runnable import RunnableLambda, RunnableSequence
# Create individual runnables (steps in the chain)
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: llm.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

chain = RunnableSequence(first=format_prompt, middle=[invoke_model], last=parse_output)

In [5]:
result = chain.invoke({"topic": "train", "joke_count": 3, "language": "Chinese"})

from IPython.display import Markdown
Markdown(result)

当然可以！以下是三个关于火车的中文笑话：

1. **笑话一：**
   - 问：火车为什么总是迟到？
   - 答：因为它总是“拖拖拉拉”（谐音“拖拖拉拉”与“火车”）。

2. **笑话二：**
   - 问：火车最喜欢什么音乐？
   - 答：当然是“咣当咣当”（火车行驶时的声音）的节奏啦！

3. **笑话三：**
   - 问：火车和飞机谁更厉害？
   - 答：当然是飞机，因为它可以“飞”，而火车只能“跑”。

希望你喜欢这些笑话！

除了RunnableSequence，还有RunnableParallel(并行)，RunnableBranch(分支)，RunnableWithMessageHistory(历史)等等结构


## 03-1 Connecting to LLM(Deepseek-chat)

In [1]:
from langchain.chat_models import init_chat_model
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

load_dotenv()
llm = init_chat_model(
    model="deepseek-chat",
    model_provider="openai",
    api_key=os.environ.get('DS_API_KEY'),
    base_url="https://api.deepseek.com",
    temperature = 0.2,
)

## 03-2 Prompt Template

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

## 03-3 Chat History

In [2]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
store = {}
def get_message_history(session_id) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

config = {"configurable": {"session_id": "a"}} # session_id is required


## 03-4 History Trimmer

In [3]:
from langchain_core.messages import trim_messages
import tiktoken

def count_tokens(messages):
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = 0
    for message in messages:
        num_tokens += len(encoding.encode(message.content))
    return num_tokens

trimmer = trim_messages(
    max_tokens=40,
    strategy="last",
    token_counter=count_tokens,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

## 03-5 Chaining Everythin with LCEL

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory

chain = trimmer | prompt | llm | StrOutputParser()

with_message_history = RunnableWithMessageHistory(chain, get_message_history)

config = {"configurable": {"session_id": "a"}}

In [7]:
from langchain_core.messages import HumanMessage
with_message_history.invoke({"messages": [HumanMessage(content="Hello, Introduce yourself")]}, config)

"Hello! I'm here to assist you with a wide range of tasks, from answering questions and providing information to helping with creative writing, problem-solving, and more. Whether you need help with research, want to brainstorm ideas, or just have a casual chat, I'm here to make your experience as smooth and enjoyable as possible. How can I assist you today?"